In [68]:
#import geopandas as gpd
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import matplotlib.pyplot as plt
import folium
import seaborn as sns
import requests
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets

In [69]:
#https://www.austintexas.gov/sites/default/files/files/Planning/Demographics/population_history_pub.pdf
population_year=[1840,1850,1860,1870,1880,1890,1900,1905,1910,1915,1920,1925,1930,1935,1940,1945,1950,1955,
                 1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,
                 1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,
                 2007,2008,2009,2010,2011,2012,2013,2014,2015,2016]
population_values=[553,629,3494,4428,11013,14575,22258,25299,29860,32368,34876,42174,53120,60082,87930,101289,
                   132459, 159502,186545,194940,201762,205394,208475,214117,218981,223981,234375,244074,251808,
                   263000,275900,290300,297500,302500,308952,321900, 331900, 341507,345890,349513,358950,373541,
                   392971,417033,447342,461039,464118,466499,465622,476447,482296,492862,508336,526128,548043,
                   567566,613458,629769, 656562,669693,680899,687708,692102,700407,718912,735088,750525,774037,
                   790390,812025,832326,855215,878733,900701,926426]

population_growth =[0,1.3,18.7,2.4,9.5,2.8,4.3,2.6,3.4,1.6,1.5,3.9,4.7,2.5,7.9,2.9,5.5,3.8,3.2,4.5,3.5,1.8,1.5,
                    2.7,2.3,2.3,4.6,4.1,3.2,4.4,4.9,5.2,2.5,1.7,2.1,4.2,3.1,2.9,1.3,1,2.7,4.1,5.2,6.1,7.3,3.1,
                    0.7,0.5,-0.2,2.3,1.2,2.2,3.1,3.5,4.2,3.6,8.1,2.7,4.3,2,1.7,1,0.6,1.2,2.6,2.3,2.1,3.1,2.1,
                    2.7,2.5,2.8,2.7,2.5,2.9]

population_data = {'Year': population_year, 'Population': population_values, 'Annual Growth Rate':population_growth}
austin_population =pd.DataFrame(data=population_data)

In [70]:
## Fazer um forecasting da população dos anos em falta  

In [71]:
austin_population.head()

,Year,Population,Annual Growth Rate
0,1840,553,0.0
1,1850,629,1.3
2,1860,3494,18.7
3,1870,4428,2.4
4,1880,11013,9.5


In [72]:
waste_aust=pd.read_csv('data/waste.csv')

## Data Exploration

In [73]:
waste_aust.shape

(740873, 8)

In [74]:
waste_aust.dtypes

Report Date      object
Load Type        object
Load Time        object
Load Weight     float64
Dropoff Site     object
Route Type       object
Route Number     object
Load ID           int64
dtype: object

In [75]:
waste_aust.columns

Index(['Report Date', 'Load Type', 'Load Time', 'Load Weight', 'Dropoff Site',
       'Route Type', 'Route Number', 'Load ID'],
      dtype='object')

In [76]:
waste_aust.isna().sum()

Report Date         0
Load Type           0
Load Time           0
Load Weight     72335
Dropoff Site        0
Route Type          0
Route Number        0
Load ID             0
dtype: int64

In [77]:
waste_aust.head()

,Report Date,Load Type,Load Time,Load Weight,Dropoff Site,Route Type,Route Number,Load ID
0,12/08/2020,BULK,12/08/2020 03:02:00 PM,5220.0,TDS LANDFILL,BULK,BU13,899097
1,12/08/2020,RECYCLING - SINGLE STREAM,12/08/2020 10:00:00 AM,11140.0,TDS - MRF,RECYCLING - SINGLE STREAM,RTAU53,899078
2,12/03/2020,RECYCLING - SINGLE STREAM,12/03/2020 10:34:00 AM,10060.0,BALCONES RECYCLING,RECYCLING - SINGLE STREAM,RHBU10,899082
3,12/07/2020,SWEEPING,12/07/2020 10:15:00 AM,7100.0,TDS LANDFILL,SWEEPER DUMPSITES,DSS04,899030
4,12/07/2020,RECYCLING - SINGLE STREAM,12/07/2020 04:00:00 PM,12000.0,TDS - MRF,RECYCLING - SINGLE STREAM,RMAU53,899048


In [78]:
waste_aust['Report Date']= pd.to_datetime(waste_aust['Report Date'], infer_datetime_format=True)
waste_aust['Load Time']= pd.to_datetime(waste_aust['Load Time'], infer_datetime_format=True)

In [79]:
waste_aust['Year'] =waste_aust['Report Date'].dt.year
waste_aust['Month'] =waste_aust['Report Date'].dt.month

In [80]:
waste_aust.head()

,Report Date,Load Type,Load Time,Load Weight,Dropoff Site,Route Type,Route Number,Load ID,Year,Month
0,2020-12-08,BULK,2020-12-08 15:02:00,5220.0,TDS LANDFILL,BULK,BU13,899097,2020,12
1,2020-12-08,RECYCLING - SINGLE STREAM,2020-12-08 10:00:00,11140.0,TDS - MRF,RECYCLING - SINGLE STREAM,RTAU53,899078,2020,12
2,2020-12-03,RECYCLING - SINGLE STREAM,2020-12-03 10:34:00,10060.0,BALCONES RECYCLING,RECYCLING - SINGLE STREAM,RHBU10,899082,2020,12
3,2020-12-07,SWEEPING,2020-12-07 10:15:00,7100.0,TDS LANDFILL,SWEEPER DUMPSITES,DSS04,899030,2020,12
4,2020-12-07,RECYCLING - SINGLE STREAM,2020-12-07 16:00:00,12000.0,TDS - MRF,RECYCLING - SINGLE STREAM,RMAU53,899048,2020,12


In [81]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [82]:
output = widgets.Output()

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(waste_aust.Year))
dropdown_routeType = widgets.Dropdown(options = unique_sorted_values_plus_ALL(waste_aust['Route Type']))
def common_filtering(year, routeType):
    output.clear_output()
    
    if (year == ALL) & (routeType == ALL):
        common_filter = waste_aust
    elif (year == ALL):
        common_filter = waste_aust[waste_aust['Route Type'] == routeType]
    elif (routeType == ALL):
        common_filter = waste_aust[waste_aust.Year == year]
    else:
        common_filter = waste_aust[(waste_aust.Year == year) & 
                                  (waste_aust['Route Type'] == routeType)]
    
    with output:
        display(common_filter)

In [83]:
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_routeType.value)
def dropdown_routeType_eventhandler(change):
    common_filtering(dropdown_year.value, change.new)

In [84]:
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_routeType.observe(dropdown_routeType, names='value')

In [85]:
display(dropdown_year)
display(dropdown_routeType)

Dropdown(options=('ALL', 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2…

Dropdown(options=('ALL', 'ALLEY BULK REMOVAL', 'BRUSH', 'BULK', 'CAP BRUSH', 'CAP BULK', 'CAP STORM', 'CAP STO…

In [86]:
display(output)

Output()

# Visualization

In [87]:
# Total de lixo recolhido por ano

In [88]:
data= waste_aust.groupby(waste_aust['Year'])[['Load Weight']].sum()
fig = px.bar(waste_aust.groupby(waste_aust['Year'])[['Load Weight']].sum())
fig.show()

In [89]:
#população por Ano 

In [90]:
fig = px.bar(x=austin_population['Year'],y=austin_population['Population'])
fig.show()

In [92]:
waste_aust.columns

Index(['Report Date', 'Load Type', 'Load Time', 'Load Weight', 'Dropoff Site',
       'Route Type', 'Route Number', 'Load ID', 'Year', 'Month'],
      dtype='object')

In [98]:
waste_aust.groupby(['Year','Load Type'])['Load Weight'].sum()

Year  Load Type                
2003  RECYCLING - SINGLE STREAM       20170.0
2004  BAGGED LITTER                    2140.0
      BRUSH                         2242000.0
      BULK                          2878116.0
      DEAD ANIMAL                     90670.0
                                      ...    
2021  RECYCLED METAL                  85670.0
      RECYCLING - SINGLE STREAM    64954430.0
      SWEEPING                      3462480.0
      TIRES                          309787.0
      YARD TRIMMING                 1504520.0
Name: Load Weight, Length: 250, dtype: float64

In [99]:
waste_aust.groupby(['Year','Month'])['Load Weight'].sum()

Year  Month
2003  1        2.017000e+04
2004  8        2.137800e+05
      9        3.578600e+05
      10       2.972507e+07
      11       3.331974e+07
                   ...     
2021  3        6.488221e+07
      4        5.088864e+07
      5        4.788044e+07
      6        4.923359e+07
      7        1.071113e+07
Name: Load Weight, Length: 205, dtype: float64

In [100]:
waste_aust.groupby(['Year','Load ID'])['Load Weight'].sum()

Year  Load ID
2003  681177     20170.0
2004  101223     11260.0
      101224      8040.0
      101225     17380.0
      101226     21260.0
                  ...   
2021  929002         0.0
      929003         0.0
      929004         0.0
      929005      4340.0
      929006      4160.0
Name: Load Weight, Length: 740820, dtype: float64

In [102]:
waste_aust.groupby(['Year','Route Number'])['Load Weight'].sum()

Year  Route Number
2003  RFBS35           20170.0
2004  BR03             41320.0
      BR05             16600.0
      BR07            170180.0
      BR13             63000.0
                        ...   
2021  YW02             79380.0
      YW03             40600.0
      YW04             71740.0
      YW05             49640.0
      YW06             21160.0
Name: Load Weight, Length: 17131, dtype: float64

In [101]:
waste_aust.groupby(['Year','Dropoff Site'])['Load Weight'].sum()

Year  Dropoff Site      
2003  BALCONES RECYCLING    2.017000e+04
2004  BARTON SKYWAY         0.000000e+00
      BURGER CENTER         8.360000e+03
      CLARKSON              0.000000e+00
      COMMERCIAL METALS     1.119000e+05
                                ...     
2021  STEINER LANDFILL      7.800000e+02
      TDS - MRF             2.910428e+07
      TDS LANDFILL          1.621761e+08
      WESTFIELD             1.602000e+04
      WILCAB                0.000000e+00
Name: Load Weight, Length: 361, dtype: float64

In [104]:
fig = px.bar(waste_aust.groupby(waste_aust['Year'])[['Load Weight']].sum())
fig.show()

## FRED

In [ ]:
waste_aust['Load Time'] = pd.to_datetime(waste_aust['Load Time'])

In [ ]:
cal = calendar()
holidays = cal.holidays(start=waste_aust['Load Time'].min(), end=waste_aust['Load Time'].max())

waste_aust['Holiday'] = waste_aust['Load Time'].isin(holidays)

In [ ]:
waste_aust['DoW'] = waste_aust['Load Time'].dt.day_name()

waste_aust.head()

In [ ]:
waste_aust.groupby('Route Number')['Load ID'].nunique()

In [ ]:
waste_aust_2019 = waste_aust.loc[(waste_aust['Load Time'] >= '2019-01-01')
                     & (waste_aust['Load Time'] < '2020-01-01')]
waste_aust_2019 = waste_aust_2019.set_index(['Load Time'])
waste_aust_2019 = waste_aust_2019.sort_index()

In [ ]:
# Weight histogram
plt.hist(waste_aust_2019['Load Weight'], bins = 20)
plt.show() 

In [ ]:
# Load Type countplot
waste_aust_2019['Load Type'].value_counts().plot(kind='barh')

In [ ]:
sns.catplot(x="DoW", y="Load Weight", kind="boxen",
            data=waste_aust_2019)

## Waste Routes

In [ ]:
waste_routes = gpd.read_file('Garbage Routes.geojson')
waste_routes.head()

In [ ]:
waste_routes.plot(figsize=(6, 6))
plt.show()

In [ ]:
m = folium.Map(location=[30.30, -97.7], zoom_start=10, tiles='CartoDB positron')

for _, r in waste_routes.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['garb_rt']).add_to(geo_j)
    geo_j.add_to(m)
m